In [ ]:
# 9/18/24 


In [ ]:
# I will make a sqlite3 database from the netflix csv file and practice some SQL queries.

# I will answer some business questions using this Netflix database.


In [1]:
import requests
import json
import os
import pandas as pd
from pandas import DataFrame, Series


In [2]:
# Helper Functions

def qry(QUERRY):
    return pd.read_sql(QUERRY, conn)


In [3]:
netflix = pd.read_csv('netflix_titles.csv')

netflix.head()


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [4]:
import sqlite3
conn = sqlite3.connect('netflix.db')
netflix.to_sql("netflix", conn, if_exists="replace", index=False)

# remember to do conn.close() at the end of your notebook


8807

In [7]:
# Check length of dataset.

total_rows = qry("""
    SELECT COUNT(*) as total_rows
    FROM netflix
    """)

total_rows


,total_rows
0,8807


In [29]:
# Check datatypes of columns.

netflix.dtypes


show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object

In [11]:
# 1. Count the Number of Movies vs TV Shows

q1 = qry("""
    SELECT type, 
        COUNT(*) as total_content
    FROM netflix
    GROUP BY type
    """)

q1


,type,total_content
0,Movie,6131
1,TV Show,2676


In [25]:
# 2. Find the Most Common Rating for Movies and TV Shows


# this is my first attempt, still some ways to go.
# q2 = qry("""
#          SELECT rating, 
#             COUNT(*) as total_content
#          FROM netflix
#          GROUP BY rating
#          ORDER BY total_content DESC
#          """)


# this is the subquery I used in the actual q2 below. 
# Running this subquery gives a good feel for the data.
# The real q2 query below just answers the question more directly and is the right way to do it.
# q2_subquery = qry("""
#          SELECT 
#             type, 
#             rating, 
#             COUNT(*), 
#             RANK() OVER(PARTITION BY type ORDER BY COUNT(*) DESC) as ranking
#          FROM netflix
#          GROUP BY 1, 2
#          ORDER BY ranking
#          """)


q2 = qry("""
         SELECT type, rating
         FROM
         (
            SELECT 
               type, 
               rating, 
               COUNT(*), 
               RANK() OVER(PARTITION BY type ORDER BY COUNT(*) DESC) as ranking
            FROM netflix
            GROUP BY 1, 2
         )
         WHERE ranking = 1
         """)

# q2_subquery

q2



,type,rating
0,Movie,TV-MA
1,TV Show,TV-MA


In [28]:
# 3. List All Movies Released in a Specific Year (e.g., 2020)

q3 = qry("""
         SELECT *
         FROM netflix
         WHERE 
            release_year = 2020 
            AND 
            type = 'Movie'
         """)

q3


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s17,Movie,Europe's Most Dangerous Man: Otto Skorzeny in ...,"Pedro de Echave García, Pablo Azorín Williams",None,None,"September 22, 2021",2020,TV-MA,67 min,"Documentaries, International Movies",Declassified documents reveal the post-WWII li...
2,s79,Movie,Tughlaq Durbar,Delhiprasad Deenadayalan,"Vijay Sethupathi, Parthiban, Raashi Khanna",None,"September 11, 2021",2020,TV-14,145 min,"Comedies, Dramas, International Movies",A budding politician has devious plans to rise...
3,s85,Movie,Omo Ghetto: the Saga,"JJC Skillz, Funke Akindele","Funke Akindele, Ayo Makun, Chioma Chukwuka Akp...",Nigeria,"September 10, 2021",2020,TV-MA,147 min,"Action & Adventure, Comedies, Dramas",Twins are reunited as a good-hearted female ga...
4,s104,Movie,Shadow Parties,Yemi Amodu,"Jide Kosoko, Omotola Jalade-Ekeinde, Yemi Blaq...",None,"September 6, 2021",2020,TV-MA,117 min,"Dramas, International Movies, Thrillers",A family faces destruction in a long-running c...
...,...,...,...,...,...,...,...,...,...,...,...,...
512,s3047,Movie,All the Freckles in the World,Yibrán Asuad,"Hánssel Casillas, Loreto Peralta, Andrea Sutto...",Mexico,"January 3, 2020",2020,TV-14,90 min,"Comedies, International Movies, Romantic Movies",Thirteen-year-old José Miguel is immune to 199...
513,s3061,Movie,Ghost Stories,"Anurag Kashyap, Dibakar Banerjee, Karan Johar,...","Janhvi Kapoor, Sobhita Dhulipala, Sukant Goel,...",India,"January 1, 2020",2020,TV-MA,145 min,"Horror Movies, International Movies, Thrillers","The directors of Emmy-nominated ""Lust Stories""..."
514,s5973,Movie,#cats_the_mewvie,Michael Margolis,None,Canada,"February 5, 2020",2020,TV-14,90 min,"Documentaries, International Movies",This pawesome documentary explores how our fel...
515,s7595,Movie,Norm of the North: Family Vacation,Anthony Bell,"Andrew Toth, Lisa Durupt, Jonathan Holmes, Pau...",United States,"May 25, 2020",2020,TV-Y7,89 min,Children & Family Movies,"Stressed by his duties as king and father, Nor..."


In [43]:
# 4. Find the Top 5 Countries with the Most Content on Netflix.
# Notice for this question, some of the country values in the dataset have multiple countries
# listed separated by commas. Need to address that (Ex: Australia, United Kingdom, Canada is 
# one of the values for country for one of the movies).


# This below is the way to do it in Postgre SQL, but I am doing it in Sqlite, which doesn't 
# have the UNNEST and STRING_TO_ARRAY functions like Postgre SQL.

# q4_subquery = qry("""
#                 SELECT
#                     UNNEST(STRING_TO_ARRAY(country, ',')) as new_country,
#                     COUNT(show_id) as total_content
#                 FROM netflix
#                 GROUP BY new_country
#                 ORDER BY total_content DESC
#                 LIMIT 5
#                 """)


# This is the best I could do with just using Sqlite. It does not account for all of the values
# that have multiple countries listed and separated by commas.
# In the cell below, I do it fully and get the full right answer, but I have to use some 
# python pandas methods to get it done.

q4 = qry("""
         SELECT 
            country, 
            COUNT(show_id)  as total_content
         FROM netflix
         GROUP BY country
         ORDER BY total_content DESC
         LIMIT 5
         """)

# q4_subquery

q4


,country,total_content
0,United States,2818
1,India,972
2,None,831
3,United Kingdom,419
4,Japan,245


In [44]:
# In this cell I actually answer question 4 fully, incorporating all the countries that are 
# separated by commas. However because I am using Sqlite and not Postgre, I have to resort
# to using some python pandas functions. But this is the full answer for the question.

q4_subquery = qry("""
                SELECT country
                FROM netflix
                """)

# Split the 'country' column into lists in Python using pandas
q4_subquery['new_country'] = q4_subquery['country'].str.split(',')

# If you want to flatten the list and unnest the values:
unnested_countries = q4_subquery.explode('new_country')

# Step 4: Count the occurrences of each country and get the top 5
top_5_countries = (unnested_countries['new_country']
                   .value_counts()
                   .head(5))

top_5_countries


United States     3211
India             1008
United Kingdom     628
 United States     479
Canada             271
Name: new_country, dtype: int64

In [47]:
# 5. Identify the Longest Movie.

q5 = qry("""
        SELECT title, duration
        FROM netflix
        WHERE type = 'Movie'
        ORDER BY CAST(SUBSTR(duration, 1, LENGTH(duration) - 4) AS INTEGER) DESC
        LIMIT 1
        """)

q5



,title,duration
0,Black Mirror: Bandersnatch,312 min


In [51]:
# 6. Find Content Added in the Last 5 Years

# q6 = qry("""
#         SELECT *
#         FROM netflix
#         WHERE TO_DATE(date_added, 'Month DD, YYYY') >= CURRENT_DATE - INTERVAL '5 years';
#         """)

# q6



In [52]:
# 7. Find All Movies/TV Shows by Director 'Rajiv Chilaka'

q7 = qry("""
         SELECT *
         FROM netflix
         WHERE director LIKE '%Rajiv Chilaka%'
         """)

q7



,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s407,Movie,Chhota Bheem - Neeli Pahaadi,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jig...",None,"July 22, 2021",2013,TV-Y7,64 min,Children & Family Movies,Things get spooky when Bheem and his buddies t...
1,s408,Movie,Chhota Bheem & Ganesh,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jig...",None,"July 22, 2021",2009,TV-Y7,68 min,Children & Family Movies,Bheem and Lord Ganesha join forces to save Dho...
2,s409,Movie,Chhota Bheem & Krishna: Mayanagari,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jig...",None,"July 22, 2021",2011,TV-Y7,67 min,Children & Family Movies,Young lord Krishna has fallen for demoness Maa...
3,s410,Movie,Chhota Bheem & Krishna: Pataliputra- City of t...,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jig...",None,"July 22, 2021",2010,TV-Y7,66 min,Children & Family Movies,Krishna needs Bheem’s help: The ruined city of...
4,s411,Movie,Chhota Bheem And The Broken Amulet,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jig...",India,"July 22, 2021",2013,TV-Y7,64 min,Children & Family Movies,When a stranger tries to steal an amulet from ...
5,s412,Movie,Chhota Bheem And The Crown of Valhalla,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jig...",None,"July 22, 2021",2013,TV-Y7,64 min,Children & Family Movies,To save the kingdom of Valhalla from invasion ...
6,s413,Movie,Chhota Bheem and the Incan Adventure,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jig...",None,"July 22, 2021",2013,TV-Y7,65 min,Children & Family Movies,"When Dholakpur’s princess is kidnapped, Bheem ..."
7,s414,Movie,Chhota Bheem and The ShiNobi Secret,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jig...",None,"July 22, 2021",2013,TV-Y7,64 min,Children & Family Movies,After learning of a samurai village under thre...
8,s415,Movie,Chhota Bheem Aur Hanuman,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jig...",None,"July 22, 2021",2012,TV-Y7,68 min,Children & Family Movies,When two evil entities kidnap a princess in a ...
9,s416,Movie,Chhota Bheem aur Krishna,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jig...",India,"July 22, 2021",2009,TV-Y7,68 min,Children & Family Movies,"Centuries after the end of a conqueror's rule,..."


In [53]:
# 8. List All TV Shows with More Than 5 Seasons

q8 = qry("""
         SELECT *
         FROM netflix
         WHERE 
            type = 'TV Show'
            AND
            CAST(SUBSTR(duration, 1, LENGTH(duration) - 8) AS INTEGER) > 5
         """)

q8


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
1,s56,TV Show,Nailed It,None,"Nicole Byer, Jacques Torres",United States,"September 15, 2021",2021,TV-PG,6 Seasons,Reality TV,Home bakers with a terrible track record take ...
2,s66,TV Show,Numberblocks,None,"Beth Chalmers, David Holt, Marcel McCalla, Ter...",United Kingdom,"September 15, 2021",2021,TV-Y,6 Seasons,Kids' TV,"In a place called Numberland, math adds up to ..."
3,s68,TV Show,Saved by the Bell,None,"Mark-Paul Gosselaar, Tiffani Thiessen, Mario L...",United States,"September 15, 2021",1994,TV-PG,9 Seasons,"Kids' TV, TV Comedies","From middle school to college, best friends Za..."
4,s83,TV Show,Lucifer,None,"Tom Ellis, Lauren German, Kevin Alejandro, D.B...",United States,"September 10, 2021",2021,TV-14,6 Seasons,"Crime TV Shows, TV Comedies, TV Dramas","Bored with being the Lord of Hell, the devil r..."
...,...,...,...,...,...,...,...,...,...,...,...,...
94,s8190,TV Show,The Andy Griffith Show,None,"Andy Griffith, Ron Howard, Don Knotts, Frances...",United States,"July 1, 2017",1967,TV-G,8 Seasons,"Classic & Cult TV, TV Comedies",Homespun humor and easygoing Sheriff Andy Tayl...
95,s8379,TV Show,The L Word,None,"Jennifer Beals, Leisha Hailey, Laurel Holloman...","Canada, United States","March 20, 2018",2009,TV-MA,6 Seasons,"Romantic TV Shows, TV Dramas",This trailblazing series follows the tangled l...
96,s8443,TV Show,The Office (U.S.),None,"Steve Carell, John Krasinski, Jenna Fischer, R...",United States,"January 1, 2016",2012,TV-14,9 Seasons,TV Comedies,This hit comedy chronicles the foibles of disg...
97,s8558,TV Show,The West Wing,None,"Martin Sheen, Rob Lowe, Allison Janney, John S...",United States,"December 25, 2015",2005,TV-14,7 Seasons,TV Dramas,This powerful political epic chronicles the tr...


In [ ]:
conn.close()
